## Etapa 1: Processamento e Consolidação dos Dados

Neste notebook, vamos ler os três arquivos JSON brutos (`vagas`, `prospects`, `applicants`), limpá-los e padronizá-los na origem, uni-los numa única base de dados e aplicar os filtros de negócio.

In [1]:
import pandas as pd
import json
import re
import os

print("--- ETAPA 1: PROCESSAMENTO E CONSOLIDAÇÃO DOS DADOS ---")

--- ETAPA 1: PROCESSAMENTO E CONSOLIDAÇÃO DOS DADOS ---


### Bloco 1: Limpeza do Arquivo de Vagas
Este bloco executa a limpeza dos títulos e dos campos de idioma do arquivo `vagas.json` e cria o `vagas_cleaned.json`.

In [2]:
def clean_vagas_json(input_path='../data/raw/vagas.json', output_path='../data/processed/vagas_cleaned.json'):
    print(f"Lendo o arquivo de vagas de: {input_path}")
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    def clean_title(title):
        # Passo 0: Tratar valores nulos (null no JSON) ou não-strings.
        if title is None or not isinstance(title, str) or not title.strip():
            return "Título Indisponível"
        
        try:
            text = str(title).strip()

            # Passo 1: Remover prefixos de código de forma segura.
            # Encontra a primeira letra e remove tudo o que vier antes se contiver um número.
            match = re.search(r'[a-zA-Z]', text)
            if match:
                first_letter_index = match.start()
                if first_letter_index > 0:
                    prefix = text[:first_letter_index]
                    if re.search(r'\d', prefix):
                        text = text[first_letter_index:]

            # Passo 2: Remover sufixos de código de forma segura.
            # Remove padrões que se parecem claramente com códigos de requisição no final.
            text = re.sub(r'\s*[-–—]\s*([A-Za-z0-9-]*\d+[A-Za-z0-9-]*)$', '', text).strip()
            # Remove também sufixos que são apenas números longos.
            text = re.sub(r'\s+\d{4,}$', '', text).strip()

            # Passo 3: Limpeza final de caracteres residuais.
            # Remove quaisquer hífens, asteriscos ou espaços que sobraram nas pontas.
            text = text.strip(' *–-_—.,:')

            # Passo 4: Garantia final contra resultado nulo ou vazio.
            # Se a limpeza removeu tudo, retorna um placeholder.
            if not text:
                return "Título Indisponível"

            return text

        except Exception as e:
            # Se qualquer passo falhar, captura a exceção e retorna um placeholder.
            return "Título Indisponível"

    total_vagas = len(data)
    print(f"Total de {total_vagas} vagas encontradas. Iniciando limpeza...")
    
    for vaga_id, vaga_details in data.items():
        # Limpeza do Título da Vaga
        if 'informacoes_basicas' in vaga_details and 'titulo_vaga' in vaga_details['informacoes_basicas']:
            original_title = vaga_details['informacoes_basicas']['titulo_vaga']
            cleaned_title = clean_title(original_title)
            vaga_details['informacoes_basicas']['titulo_vaga'] = cleaned_title

        # Limpeza e Padronização dos Níveis de Idioma
        if 'perfil_vaga' in vaga_details:
            # Inglês
            nivel_ingles = vaga_details['perfil_vaga'].get('nivel_ingles')
            if nivel_ingles is None or str(nivel_ingles).strip() == '':
                vaga_details['perfil_vaga']['nivel_ingles'] = 'Nenhum'
            else:
                vaga_details['perfil_vaga']['nivel_ingles'] = str(nivel_ingles).strip()
            
            # Espanhol
            nivel_espanhol = vaga_details['perfil_vaga'].get('nivel_espanhol')
            if nivel_espanhol is None or str(nivel_espanhol).strip() == '':
                vaga_details['perfil_vaga']['nivel_espanhol'] = 'Nenhum'
            else:
                vaga_details['perfil_vaga']['nivel_espanhol'] = str(nivel_espanhol).strip()

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    print(f"Limpeza concluída. Arquivo salvo em: {output_path}")

# Executa a função
clean_vagas_json()

Lendo o arquivo de vagas de: ../data/raw/vagas.json
Total de 14081 vagas encontradas. Iniciando limpeza...
Limpeza concluída. Arquivo salvo em: ../data/processed/vagas_cleaned.json


### Bloco 2: Carregar `vagas_cleaned.json` para DataFrame

In [3]:
print("\n--- Bloco 2: Estruturando vagas com método padronizado ---")

# Carrega o arquivo JSON de vagas já limpo
with open('../data/processed/vagas_cleaned.json', 'r', encoding='utf-8') as f:
    vagas_data = json.load(f)

# Converte o dicionário para um DataFrame com o ID como índice e colunas aninhadas
vagas_raw_df = pd.DataFrame.from_dict(vagas_data, orient='index')
vagas_raw_df.index.name = 'id_vaga'

# Define as colunas aninhadas que serão achatadas (mesma abordagem dos applicants)
nested_columns = ['informacoes_basicas', 'perfil_vaga', 'beneficios']
df_parts = [vagas_raw_df.drop(columns=nested_columns, errors='ignore')]

# Itera sobre cada coluna aninhada, normaliza e adiciona o prefixo
for col in nested_columns:
    if col in vagas_raw_df.columns:
        # Garante que a coluna não contenha apenas valores nulos antes de normalizar
        series_no_na = vagas_raw_df[col].dropna()
        if not series_no_na.empty and isinstance(series_no_na.iloc[0], dict):
            # Normaliza a seção e adiciona o nome da seção como prefixo
            normalized_part = pd.json_normalize(vagas_raw_df[col]).add_prefix(f"{col}.")
            normalized_part.index = vagas_raw_df.index
            df_parts.append(normalized_part)

# Concatena as partes (colunas originais + colunas achatadas)
vagas_df = pd.concat(df_parts, axis=1)
vagas_df.reset_index(inplace=True)

print(f"DataFrame de vagas criado com {vagas_df.shape[0]} linhas e {vagas_df.shape[1]} colunas.")

# Exibe as primeiras colunas para verificação do novo padrão
print("\nAmostra dos nomes das colunas:")
print(vagas_df.columns.tolist()[:15])


--- Bloco 2: Estruturando vagas com método padronizado ---
DataFrame de vagas criado com 14081 linhas e 45 colunas.

Amostra dos nomes das colunas:
['id_vaga', 'informacoes_basicas.data_requicisao', 'informacoes_basicas.limite_esperado_para_contratacao', 'informacoes_basicas.titulo_vaga', 'informacoes_basicas.vaga_sap', 'informacoes_basicas.cliente', 'informacoes_basicas.solicitante_cliente', 'informacoes_basicas.empresa_divisao', 'informacoes_basicas.requisitante', 'informacoes_basicas.analista_responsavel', 'informacoes_basicas.tipo_contratacao', 'informacoes_basicas.prazo_contratacao', 'informacoes_basicas.objetivo_vaga', 'informacoes_basicas.prioridade_vaga', 'informacoes_basicas.origem_vaga']


### Bloco 3: Processar `prospects.json`

In [4]:
print("\nProcessando prospects.json...")
with open('../data/raw/prospects.json', 'r', encoding='utf-8') as f:
    prospects_data = json.load(f)

prospects_list = []
for vaga_id, data in prospects_data.items():
    for prospect in data.get('prospects', []):
        if prospect: 
            prospect_info = {
                'id_vaga': vaga_id,
                **prospect
            }
            prospects_list.append(prospect_info)

prospects_df = pd.DataFrame(prospects_list).add_prefix('prospect.')
prospects_df.rename(columns={'prospect.id_vaga': 'id_vaga'}, inplace=True)
print(f"prospects.json processado. {len(prospects_df)} candidaturas encontradas.")


Processando prospects.json...
prospects.json processado. 53759 candidaturas encontradas.


### Bloco 4: Processar `applicants.json`

In [ ]:
print("\nProcessando applicants.json...")
applicants_raw_df = pd.read_json('../data/raw/applicants.json', orient='index')
applicants_raw_df.index.name = 'id_candidato'

# Limpeza e padronização dos níveis de idioma (antes do flatten)
def clean_applicant_languages(row):
    if isinstance(row['formacao_e_idiomas'], dict):
        formacao = row['formacao_e_idiomas'].copy()
        for lang in ['nivel_ingles', 'nivel_espanhol']:
            nivel = formacao.get(lang)
            if nivel is None or not str(nivel).strip():
                formacao[lang] = 'Nenhum'
            else:
                formacao[lang] = str(nivel).strip()
        return formacao
    return row['formacao_e_idiomas']

if 'formacao_e_idiomas' in applicants_raw_df.columns:
    applicants_raw_df['formacao_e_idiomas'] = applicants_raw_df.apply(clean_applicant_languages, axis=1)
    print("Níveis de idioma dos applicants foram limpos e padronizados (antes do flatten).")
else:
    print("Aviso: Coluna 'formacao_e_idiomas' não encontrada. A limpeza não foi aplicada.")

# Achatamento (flattening) dos dados do candidato
nested_columns = [
    'infos_basicas', 'informacoes_pessoais', 'informacoes_profissionais', 
    'formacao_e_idiomas', 'cargo_atual'
]
df_parts = [applicants_raw_df.drop(columns=nested_columns, errors='ignore')]

for col in nested_columns:
    if col in applicants_raw_df.columns:
        series_no_na = applicants_raw_df[col].dropna()
        if not series_no_na.empty and isinstance(series_no_na.iloc[0], dict):
            normalized_part = pd.json_normalize(applicants_raw_df[col]).add_prefix(f"{col}.")
            normalized_part.index = applicants_raw_df.index
            df_parts.append(normalized_part)

applicants_df = pd.concat(df_parts, axis=1)
applicants_df.reset_index(inplace=True)
print("Dados dos candidatos foram achatados (flattened).")

print(f"Total de candidatos que permanecem na base: {len(applicants_df)}")

print("\nGuardando applicants_cleaned.json ---")
APPLICANTS_PROCESSED_PATH = '../data/processed/applicants_cleaned.json'

# Para guardar no mesmo formato original (JSON com ID como chave), definimos o ID como índice
applicants_to_save_df = applicants_df.set_index('id_candidato')
applicants_to_save_df.to_json(APPLICANTS_PROCESSED_PATH, orient='index', indent=4, force_ascii=False)

print(f"arquivo '{APPLICANTS_PROCESSED_PATH}' guardado com sucesso.")


Processando applicants.json...
Níveis de idioma dos applicants foram limpos e padronizados (antes do flatten).
Dados dos candidatos foram achatados (flattened).
Total de candidatos que permanecem na base: 42482

Guardando applicants_cleaned.json ---
arquivo '../data/processed/applicants_cleaned.json' guardado com sucesso.


### Bloco 5: Unificar as Bases

In [6]:
print("\nUnificando os dados...")
# Garante que as chaves de união sejam do mesmo tipo (string)
prospects_df['id_vaga'] = prospects_df['id_vaga'].astype(str)
vagas_df['id_vaga'] = vagas_df['id_vaga'].astype(str)
prospects_df['prospect.codigo'] = prospects_df['prospect.codigo'].astype(str)
applicants_df['id_candidato'] = applicants_df['id_candidato'].astype(str)

dados_consolidados = pd.merge(prospects_df, vagas_df, on='id_vaga', how='left')
dados_consolidados = pd.merge(dados_consolidados, applicants_df, left_on='prospect.codigo', right_on='id_candidato', how='inner')
print("Unificação concluída.")


Unificando os dados...
Unificação concluída.


### Bloco 6: Criar a Variável-Alvo (`match`)

In [7]:
print("\nCriando a variável-alvo 'match'...")
dados_consolidados['match'] = (dados_consolidados['prospect.situacao_candidado'] == 'Contratado pela Decision').astype(int)
print("Variável-alvo criada.")


Criando a variável-alvo 'match'...
Variável-alvo criada.


### Bloco Final: Análise e Salvamento

In [8]:
print("\n--- BASE DE DADOS CONSOLIDADA ---")
print(f"Dimensões da base final: {dados_consolidados.shape[0]} linhas, {dados_consolidados.shape[1]} colunas")

print("\nDistribuição da variável-alvo:")
print(dados_consolidados['match'].value_counts())

dados_consolidados.to_json('../data/processed/dados_consolidados.json', orient='records', lines=True, force_ascii=False)
print("\nArquivo '../data/processed/dados_consolidados.json' salvo com sucesso!")
print("\n--- ETAPA 1 CONCLUÍDA ---")


--- BASE DE DADOS CONSOLIDADA ---
Dimensões da base final: 45095 linhas, 111 colunas

Distribuição da variável-alvo:
match
0    42840
1     2255
Name: count, dtype: int64

Arquivo '../data/processed/dados_consolidados.json' salvo com sucesso!

--- ETAPA 1 CONCLUÍDA ---
